In [26]:
import numpy as np
import pandas as pd
import random
from utils.consts import level_order
import math

In [27]:
static_path = '/home/mikolaj/git/pos-prediction-graph/csv_files/static_with_t.csv'
static_df = pd.read_csv(static_path)
#drop column "number"
# static_df = static_df.drop(columns=['number'])

In [28]:
static_df

,name,number,x,y,z,parent,distance_to_parent
0,root,0,0.0,0.0,0.0,0,0
1,root_cross_right,-1,10.0,1.0,0.0,root,10
2,root_cross_right2,-1,12.0,-15.0,0.0,root_cross_right,16
3,root_cross_left,-1,-13.0,-20.0,0.0,root,24
4,conn_a,104,20.0,-50.0,-9.0,root_cross_right,53
5,cross_e,-1,-13.0,-25.0,0.0,root_cross_left,5
6,cross_e_c,-1,-4.0,-55.0,-10.0,cross_e,33
7,cross_f,-1,-13.0,-30.0,0.0,cross_e,5
8,cross_k,-1,-13.0,-50.0,0.0,cross_f,20
9,cross_f30,-1,-3.0,-50.0,0.0,cross_f,22


In [29]:
def calc_child_pos(parent_cords: tuple, max_distance: float):
    current_pos = parent_cords
    direction = (
        random.uniform(-1, 1),
        random.uniform(-1, -.6),
        random.uniform(-1, 1),
    )
    length = math.sqrt(sum([x**2 for x in direction]))
    direction = tuple([x / length for x in direction])

    distance = random.uniform(max_distance*.8, max_distance)

    new_pos = tuple([current_pos[i] + direction[i] * distance for i in range(3)])
    new_pos = np.round(new_pos, 2)
    return new_pos

In [30]:
skip_names = [
    "root_cross_right",
    "root_cross_left",
    "root_cross_right2",
]

In [31]:
for node_name in level_order:
    if node_name in skip_names:
        continue
    node_df = static_df[static_df['name'] == node_name]
    parent_df = static_df[static_df['name'] == node_df['parent'].iloc[0]]
    parent_cords = (parent_df['x'].iloc[0], parent_df['y'].iloc[0], parent_df['z'].iloc[0])
    new_pos = calc_child_pos(parent_cords, node_df['distance_to_parent'].iloc[0])
    static_df.loc[static_df['name'] == node_name, ['x', 'y', 'z']] = new_pos
    

In [32]:
import networkx as nx
from utils.graph_utils import plot_graph
import numpy as np
import pandas as pd

G1 = nx.Graph()
for i in range(len(static_df)):
    G1.add_node(static_df["name"][i], pos=(static_df["x"][i], static_df["z"][i], static_df["y"][i]))
for i in range(len(static_df)):
    if static_df["parent"][i] != 0 and static_df["parent"][i] != "0":
        G1.add_edge(static_df["name"][i], static_df["parent"][i])


plot_graph(static_graph=G1, detected_graph=None, name="random", save=True)

Graph saved to random.html


In [33]:
static_df = static_df[static_df['number'] >= 0]
static_df

,name,number,x,y,z,parent,distance_to_parent
0,root,0,0.00,0.00,0.00,0,0
4,conn_a,104,11.91,-26.43,-35.48,root_cross_right,53
10,conn_f_30,30,-13.13,-69.58,-8.28,cross_f30,39
12,conn_f_37,37,38.42,-89.23,11.21,cross_f37,14
13,conn_f_60,60,-10.24,-92.14,-29.76,box,53
14,conn_e,10,2.86,-59.45,19.52,cross_e_c,32
15,conn_c,14,-13.96,-62.53,14.65,cross_e_c,32
16,bag_b_l,55,-12.57,-89.16,-10.16,box,51
17,box,53,-7.36,-48.02,-0.77,cross_k,20
18,cross_gh,80,4.38,-75.17,-22.37,box,38


In [34]:
#save
static_df.to_csv("/home/mikolaj/git/pos-prediction-graph/csv_files/mega_randomized.csv", index=False)

In [35]:
STATIC_PATH = "/home/mikolaj/git/pos-prediction-graph/csv_files/static_with_t.csv"
static_df = pd.read_csv(STATIC_PATH)
skip_names = [
"root_cross_right",
"root_cross_left",
"root_cross_right2",
]



for node_name in level_order:
    if node_name in skip_names:
        continue
    node_df = static_df[static_df["name"] == node_name]
    parent_name = node_df["parent"].iloc[0]
    parent_df = static_df[static_df["name"] == parent_name]
    print(static_df)
    parent_cords = (
        parent_df["x"].iloc[0],
        parent_df["y"].iloc[0],
        parent_df["z"].iloc[0],
    )
    new_pos = calc_child_pos(parent_cords, node_df["distance_to_parent"].iloc[0])
    static_df.loc[static_df["name"] == node_name, ["x", "y", "z"]] = new_pos


                 name  number     x      y     z             parent  \
0                root       0   0.0    0.0   0.0                  0   
1    root_cross_right      -1  10.0    1.0   0.0               root   
2   root_cross_right2      -1  12.0  -15.0   0.0   root_cross_right   
3     root_cross_left      -1 -13.0  -20.0   0.0               root   
4              conn_a     104  20.0  -50.0  -9.0   root_cross_right   
5             cross_e      -1 -13.0  -25.0   0.0    root_cross_left   
6           cross_e_c      -1  -4.0  -55.0 -10.0            cross_e   
7             cross_f      -1 -13.0  -30.0   0.0            cross_e   
8             cross_k      -1 -13.0  -50.0   0.0            cross_f   
9           cross_f30      -1  -3.0  -50.0   0.0            cross_f   
10          conn_f_30      30  12.0  -86.0   0.0          cross_f30   
11          cross_f37      -1   7.0 -105.0  -9.0          cross_f30   
12          conn_f_37      37  17.0 -115.0  -9.0          cross_f37   
13    

In [36]:
static_df[static_df["name"] == parent_name]

,name,number,x,y,z,parent,distance_to_parent
18,cross_gh,80,-9.84,-80.83,-15.83,box,38
